##### https://medium.com/@erikhallstrm/hello-world-rnn-83cd7105b767

In [1]:
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

num_epochs = 10
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5

In [2]:
def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)

In [3]:
def generateData(data_length):
    x = np.array(np.random.choice(2, data_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

In [4]:
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

init_state = tf.placeholder(tf.float32, [batch_size, state_size])

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

# Unpack columns
inputs_series = tf.split(batchX_placeholder, num_or_size_splits=truncated_backprop_length, axis=1)
labels_series = tf.unstack(batchY_placeholder, axis=1)

# Forward passes
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=state_size, activation=tf.nn.tanh)
states_series, current_state = tf.nn.static_rnn(cell, inputs_series, initial_state=init_state)

logits_series = [tf.matmul(state, W2) + b2 for state in states_series]
predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logits_series,labels_series)]
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

In [5]:
data_length = 50000
x,y = generateData(data_length)

In [6]:
num_batches = data_length//batch_size//truncated_backprop_length

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    plt.ion()
    plt.figure()
    plt.show()
    loss_list = []

    for epoch_idx in range(num_epochs):
        _current_state = np.zeros((batch_size, state_size))
        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    init_state:_current_state
                })

            loss_list.append(_total_loss)

        print("Epoch ", epoch_idx, "Loss", _total_loss)
        
    # Generate some new data and see if it can recreate Y
    data_length = 100
    x,y = generateData(data_length)
    batchX = x[:,:truncated_backprop_length]
    batchY = y[:,:truncated_backprop_length]
    preds = sess.run(predictions_series, feed_dict={
        batchX_placeholder:batchX,
        init_state:_current_state})
        #plot(loss_list, _predictions_series, batchX, batchY)

plt.ioff()
plt.show()

<Figure size 432x288 with 0 Axes>

Epoch  0 Loss 0.0015892227
Epoch  1 Loss 0.0012222348
Epoch  2 Loss 0.0007174706
Epoch  3 Loss 0.0005064256
Epoch  4 Loss 0.00038730566
Epoch  5 Loss 0.00031118758
Epoch  6 Loss 0.0002586983
Epoch  7 Loss 0.00022051121
Epoch  8 Loss 0.00019163868
Epoch  9 Loss 0.00016908767


In [9]:
# See if "preds" matches "batchY". Ignore the first 3 indeces: Y has no "X" to be based on
idx = int(input("Enter an index value"))

array([[1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0],
       [1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1],
       [1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1],
       [1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1]])

In [10]:
print(batchY[:,idx])
print(np.argmax(preds[idx],axis=1))

array([[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1],
       [0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0]])